In [1]:
import os
import zipfile
import random
import shutil
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
from os import getcwd
import numpy as np

In [2]:
path_cats_and_dogs = f"{getcwd()}\cats-vs-dogs.zip"

local_zip = path_cats_and_dogs
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall(f'{getcwd()}')
zip_ref.close()

In [3]:
print(len(os.listdir(f'{getcwd()}\cats-vs-dogs\cats')))
print(len(os.listdir(f'{getcwd()}\cats-vs-dogs\dogs')))


5001
5001


In [4]:
try:
    os.mkdir(f'{getcwd()}\cats-dogs')
    os.mkdir(f'{getcwd()}\cats-dogs\\training')
    os.mkdir(f'{getcwd()}\cats-dogs\\testing')
    os.mkdir(f'{getcwd()}\cats-dogs\\training\\cats')
    os.mkdir(f'{getcwd()}\cats-dogs\\training\\dogs')
    os.mkdir(f'{getcwd()}\cats-dogs\\testing\\cats')
    os.mkdir(f'{getcwd()}\cats-dogs\\testing\\dogs')
except OSError:
    print('wrong')

In [5]:

def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):   
    src=SOURCE
    all_file_names=os.listdir(src)
    random.sample(all_file_names,len(all_file_names))
    train_FileNames,test_FileNames=np.split(np.array(all_file_names),
                             [int(len(all_file_names)*SPLIT_SIZE)]
                             )
    
    train_FileNames = [src+'\\'+ name for name in train_FileNames.tolist()]
    test_FileNames = [src+'\\' + name for name in test_FileNames.tolist()]

    
    for names in train_FileNames:
        shutil.copy(names,TRAINING)
        
    for names in test_FileNames:
        shutil.copy(names,TESTING)


CAT_SOURCE_DIR = f'{getcwd()}\cats-vs-dogs\cats'
TRAINING_CATS_DIR = f'{getcwd()}\cats-dogs\\training\\cats'
TESTING_CATS_DIR = f'{getcwd()}\cats-dogs\\testing\\cats'
DOG_SOURCE_DIR = f'{getcwd()}\cats-vs-dogs\dogs'
TRAINING_DOGS_DIR = f'{getcwd()}\cats-dogs\\training\\dogs'
TESTING_DOGS_DIR = f'{getcwd()}\cats-dogs\\testing\\dogs'

split_size = .9
split_data(CAT_SOURCE_DIR, TRAINING_CATS_DIR, TESTING_CATS_DIR, split_size)
split_data(DOG_SOURCE_DIR, TRAINING_DOGS_DIR, TESTING_DOGS_DIR, split_size)

In [6]:
print(len(os.listdir(f'{getcwd()}\cats-dogs\\training\\cats')))
print(len(os.listdir(f'{getcwd()}\cats-dogs\\training\\dogs')))
print(len(os.listdir(f'{getcwd()}\cats-dogs\\testing\\cats')))
print(len(os.listdir(f'{getcwd()}\cats-dogs\\testing\\dogs')))


4500
4500
501
501


In [ ]:

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16,(3,3), activation='relu', input_shape=(150,150,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dense(units=1, activation='sigmoid')
    
])

model.compile(optimizer=RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['acc'])

In [ ]:
TRAINING_DIR =f'{getcwd()}\cats-dogs\\training' 
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest")
    

train_generator = train_datagen.flow_from_directory(
    TRAINING_DIR,
    batch_size=32,
    target_size=(150,150),
    class_mode='binary'
)

VALIDATION_DIR = f'{getcwd()}\cats-dogs\\testing'
validation_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)


validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_DIR,
    batch_size=32,
    target_size=(150,150),
    class_mode='binary'
)



In [ ]:
history = model.fit_generator(train_generator,
                              epochs=25,
                              verbose=1,
                              validation_data=validation_generator)


In [ ]:
# PLOT LOSS AND ACCURACY
%matplotlib inline

import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

acc=history.history['acc']
val_acc=history.history['val_acc']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot(epochs, loss, 'r', "Training Loss")
plt.plot(epochs, val_loss, 'b', "Validation Loss")


plt.title('Training and validation loss')
